# DEEP LEARNING 

# Partie sur les images

### Objectif à dépasser : weighted F1-score = 0.5534 (Resnet)
avec CNN/Transfer learning

Import libraries:

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds\\images')

Récupérer les données:

In [2]:
X = pd.read_csv('X_train_update.csv',index_col=0)
y = pd.read_csv('Y_train_CVw08PX.csv',index_col=0)

Liste des images:

In [20]:
os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds')

In [21]:
import os #Miscellaneous operating system interfaces
#https://docs.python.org/3/library/os.html

#get current working directory
current_path = os.getcwd() 

#Training images path
images_path = current_path + r'/images/image_train - Old/'

#List with the name of all training images
images_list = os.listdir(images_path)

Géneration nom des images:

In [22]:
#Create a column with the name of the picture
X['image name'] = 'image_' + X['imageid'].map(str) + '_product_' + X['productid'].map(str) + '.jpg'
X['image name']

0        image_1263597046_product_3804725264.jpg
1         image_1008141237_product_436067568.jpg
2          image_938777978_product_201115110.jpg
3           image_457047496_product_50418756.jpg
4         image_1077757786_product_278535884.jpg
                          ...                   
84911      image_941495734_product_206719094.jpg
84912    image_1188462883_product_3065095706.jpg
84913     image_1009325617_product_440707564.jpg
84914    image_1267353403_product_3942400296.jpg
84915       image_684671297_product_57203227.jpg
Name: image name, Length: 84916, dtype: object

### Répartition des images dans les échantillons train, validation et test

Répartion :

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 123)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 123)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1, random_state = 123)

X_val = X_train.iloc[:2400,:]
y_val = y_train.iloc[:2400,:]

X_train = X_train.iloc[:21600,:]
y_train = y_train.iloc[:21600:,:]

X_test = X_test.iloc[:6000,:]
y_test = y_test.iloc[:6000,:]

In [24]:
X_train["class"] = y_train
X_test["class"] = y_test
X_val["class"] = y_val

X_train["class"] = X_train["class"].astype('str')
X_test["class"] = X_test["class"].astype('str')
X_val["class"] = X_val["class"].astype('str')

Qualité de la répartition

trainy = pd.DataFrame(y_train.value_counts(), columns = ['Nombre_images_train'])
testy = pd.DataFrame(y_test.value_counts(), columns = ['Nombre_images_test'])
train_test = trainy.merge(testy, right_index = True, left_index = True)
train_test['y'] = train_test.index
yval = []
for i in train_test['y']:
    yval.append(i[0])
train_test['y'] = yval
train_test

In [25]:
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import preprocess_input


train_data_generator = ImageDataGenerator(rescale = 1./255, 
        preprocessing_function = preprocess_input,  # data augmentation
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 1.1,
        horizontal_flip = True
        )

val_data_generator = ImageDataGenerator(rescale = 1./255, 
        preprocessing_function = preprocess_input,  # data augmentation
        rotation_range = 10,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 1.1,
        horizontal_flip = True
        )

test_data_generator = ImageDataGenerator(rescale = 1./255, 
    preprocessing_function = preprocess_input)

In [29]:
batch_size = 32
path = os.chdir('C:\\Users\\barry\\OneDrive - CSTBGroup\\image_ds\\images\\image_train')
X_train["class"] = X_train["class"].astype(str)
X_test["class"] = X_test["class"].astype(str)
X_val["class"] = X_val["class"].astype(str)

train_generator = train_data_generator.flow_from_dataframe(dataframe=X_train,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                          target_size = (150,150), 
                                                          batch_size = batch_size)


val_generator = val_data_generator.flow_from_dataframe(dataframe=X_val,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                           target_size = (150, 150), 
                                                          batch_size = batch_size)

test_generator = test_data_generator.flow_from_dataframe(dataframe=X_test,
                                                          directory=path,
                                                           x_col = "image name",
                                                           y_col = "class",
                                                           class_mode ="sparse",
                                                           target_size = (150, 150), 
                                                          batch_size = batch_size, shuffle = False)

Found 21600 validated image filenames belonging to 27 classes.
Found 2400 validated image filenames belonging to 27 classes.
Found 6000 validated image filenames belonging to 27 classes.


In [30]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

In [36]:
import tensorflow as tf

In [57]:
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense, GlobalAveragePooling2D
from keras import layers

x = layers.Flatten()(base_model.output)
x = layers.Dense(8**4, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(27, activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])


In [58]:
epoch = 2

inc_history = model.fit_generator(train_generator, 
                                epochs = epoch,
                                steps_per_epoch = len(X_train)//batch_size,
                                validation_data = val_generator,
                                validation_steps = len(X_val)//batch_size)

C:\Users\barry\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
675/675 [==============================] - 1043s 2s/step - loss: 3.0212 - acc: 0.1325 - val_loss: 2.8496 - val_acc: 0.1675
Epoch 2/2
675/675 [==============================] - 1133s 2s/step - loss: 2.8766 - acc: 0.1686 - val_loss: 2.7997 - val_acc: 0.1879


Entrainement X derniers layers freezed :

In [59]:
X = 3
for layer in base_model.layers[-X:]:
    layer.trainable = True

In [60]:
model.compile(optimizer = RMSprop(lr=0.0001), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])


In [61]:
inc_history_defreeze = model.fit(train_generator, 
                                epochs = epoch,
                                steps_per_epoch = len(X_train)//batch_size,
                                validation_data = val_generator,
                                validation_steps = len(X_val)//batch_size
                                )

Epoch 1/2
675/675 [==============================] - 1213s 2s/step - loss: 2.8331 - acc: 0.1848 - val_loss: 2.7777 - val_acc: 0.1825
Epoch 2/2
675/675 [==============================] - 1154s 2s/step - loss: 2.7909 - acc: 0.1972 - val_loss: 2.7388 - val_acc: 0.2208


Evaluation du modèle :

In [62]:
y_inception_pred_2 = model.predict_generator(test_generator, verbose = 1)
y_inception_pred_2 = y_inception_pred_2.argmax(axis = 1)

C:\Users\barry\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


188/188 [==============================] - 143s 739ms/step


F1_score : 

In [63]:
from sklearn.metrics import f1_score
f1_score(test_generator.classes, y_inception_pred_2, average = "weighted")

0.19981439633302944

In [ ]:
liste_epoch = []
liste_score = []

Matrice de confusion :

In [44]:
from sklearn.metrics import confusion_matrix
print(pd.DataFrame(confusion_matrix( test_generator.classes, y_inception_pred_2)))

    0   1    2   3   4   5    6   7   8   9   ...  17  18  19   20  21  22  \
0   27   0   30   0   4   0    5   0   0   0  ...   0   0   0   13   0  34   
1    2   1   11   0  21   0   29   0   0   0  ...   0   1   0   59   2   5   
2    5   0  149   0   2   0    3   0   0   0  ...   0   0   0   10   0  17   
3    0   0    4   0   3   0    6   0   0   0  ...   0   0   0   12   0   2   
4    3   0   14   0  49   0   65   0   2   2  ...   0   1   0   94   0   8   
5    0   0   10   0  18   0   16   0   0   1  ...   0   1   0   25   1  10   
6    0   0    7   0  12   0  170   0   0   1  ...   0   0   0   85   2   3   
7    0   0    1   0   2   0    9   0   1   1  ...   0   1   0   30   1   1   
8    1   0    3   0  13   0   64   0   1   3  ...   0   0   0   51   0   2   
9    0   0    5   0  13   0   24   0   2   9  ...   0   1   0   79   0   2   
10   1   0    8   0  12   0   37   0   0   3  ...   0   2   0   98   1   2   
11   1   0   11   0   5   0   32   0   0   2  ...   0   2   0   